In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

ModuleNotFoundError: No module named 'torch'

In [38]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
# max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split, device):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model, device):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

cuda


In [40]:
def model_train_test (max_iters, device):

    m = BigramLanguageModel(vocab_size).to(device)

    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

    for iter in range(max_iters):

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0:
            losses = estimate_loss(m, device)
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # sample a batch of data
        xb, yb = get_batch('train', device)
        # xb, yb = xb.to(device), yb.to(device)
        

        # evaluate the loss
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        if iter % 100 == 0:
            print(f"epoch[{iter}]: loss={loss.item()}")

    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

model_train_test(100, device)

step 0: train loss 4.7154, val loss 4.7080
epoch[0]: loss=4.748942852020264

k'nNE:FPAkULJKg..tYoYq'WGhgwkyom,
brSB;e WsV.-s?yomgFVqqrS'eFlxtaJRY;$$OzrZjWhNGy
nH:
ugUbxIERTi',
c$$jfsom&BsSfR&cXsgl.mt
IEiaroGJmmVpDSjTs.xseueHpPeABy
Y:ovetL!Z?AB,?PcLwowoTsqIkpovetKH.TeistJumqIAsWxV;'eRz-a,LU:yrD$jOC,
ng'kPkA?ieuUB,-lGh Atve,FsoxthqjThJE&B'beakwuOp
FvzhvE&
Nwyam&k ! .vDCSU;lHnF',
WZ!QNCllcHIRzPr vy,OTFPe
LIFGyoA?VTItidk
bLw$j&XnGendLzXC,?LFy
keHLIUXqoYKeiuG
NtQe nLI!vee A3RKGSxRpDytGyon wubeC$ufsirthW$u,fstE&BlWolbkm,PonHA hhO
FPryoab.cTjwrtch&LXvY$jxttTirWfG!u;qyoekWog?Pee


In [17]:
model = BigramLanguageModel(vocab_size).to(device)

In [18]:
print(model)

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 65)
)


# torchviz

In [19]:
from torchviz import make_dot
xb, yb = get_batch('train')
xb, yb = xb.to(device), yb.to(device)
logits, loss = model(xb, yb)
make_dot(loss, params=dict(list(model.named_parameters()))).render("model_torchviz", format="png")

'model_torchviz.png'

# HiddenLayer

In [20]:
import hiddenlayer as hl

In [22]:
model_hl = BigramLanguageModel(vocab_size).to(device)
x, y = get_batch('train')
hl.build_graph(model_hl, (x,y) )


AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'